In [1]:
import nltk

# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

from spacy_download import load_spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('data/post_covidTweets_v2.csv')
print(df.username.unique())
df.head()
df.count()

['RochelleAdame7' 'brian_tietz' 'LarryBoorstein' 'VigilantFox'
 'Mrmangajojo' 'HawkInATX' 'vindicator1337' 'KoonsKarl' 'JohnnyRode'
 'Oneguy83440339' 'Excuseeeme' 'StuckInSanDiego' 'JudithArnett1'
 'Tigerlily1011' 'Deevoee' 'Katetheskates' 'LankeyPants250'
 'FellwockPerry' 'most_articulate' 'Smpwns' 'USBornNRaised' 'AlisynGayle'
 'RobinBr83683748' 'urbiasdqsu' 'adelaidians' 'mrmikebones' '_LoveLiberty'
 'LonelyFreedom' 'ClimateDepot' 'CharmsYahoo' 'STRAT777' 'Salty_shifter'
 'JamieO123' 'Faketriots' 'CristinoCastro9' '76Americanwomen'
 'io_aircraft_inc' 'MrMarchi' 'KingGrunt' 'MrSkanner' 'PatrioticPatty2'
 'SherryHighHorse' 'DailyReformerUS' 'TruckCastreauJr' 'valentino_nora'
 'OrmeStephan' 'JohnT43838185' 'Kellyeod' 'DennisOgden007' 'Alec121212'
 'JackHay01988146' 'small4lyfe' 'Allison86435394' 'nomandates23'
 'andriedeja' 'ImmunifyLife' 'tomcoates' 'caz_sampson' 'Crof'
 'Amfirstnewswire' 'WoodyWhitcomb' 'GeorgeG55508748' 'livejelly'
 'BuzzerlyApp' 'kyle8824' 'stephphilip8' 'infamous_

datetime          40886
tweet_id          40886
text              40886
username          40886
url               40886
user_location     27639
like_count        40886
retweet_count     40886
follower_count    40886
reply_count       40886
verified          40886
hashtags          14700
dtype: int64

In [9]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

#remove URL
data = [re.sub('http://\S+|https://\S+', '', sent) for sent in data]
data = [re.sub('http[s]?://\S+', '', sent) for sent in data]
data = [re.sub(r"http\S+", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Did I say it was evil? Did I say dont get it? NEWSFLASH....I DIDNT. And I '
 'get the flu shot every year and my mammogram in the same week until '
 'recently. I have never been told I had to postpone my mammogram for any '
 'other vaccine. NOT ONE.']


In [10]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['did', 'say', 'it', 'was', 'evil', 'did', 'say', 'dont', 'get', 'it', 'newsflash', 'didnt', 'and', 'get', 'the', 'flu', 'shot', 'every', 'year', 'and', 'my', 'mammogram', 'in', 'the', 'same', 'week', 'until', 'recently', 'have', 'never', 'been', 'told', 'had', 'to', 'postpone', 'my', 'mammogram', 'for', 'any', 'other', 'vaccine', 'not', 'one']]


In [11]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['did', 'say', 'it', 'was', 'evil', 'did', 'say', 'dont', 'get', 'it', 'newsflash', 'didnt', 'and', 'get', 'the', 'flu', 'shot', 'every', 'year', 'and', 'my', 'mammogram', 'in', 'the', 'same', 'week', 'until', 'recently', 'have', 'never', 'been', 'told', 'had', 'to', 'postpone', 'my', 'mammogram', 'for', 'any', 'other', 'vaccine', 'not', 'one']


In [12]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Will download the model if it isn't installed yet 
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
vocab = len(id2word.token2id)

pd.Series([t for tokens in data_words_bigrams for t in tokens if "_" in t]).unique()

In [ ]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



In [ ]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



In [ ]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis